## Key words
### 기술통계랑, 가설검정, 군집분석

### 1. 지선, 간선 버스의 경우 노선당 년 수익이 10억 이하인 경우 배차간격 조정이나 노선 변경 등 수익구조 개선이 필요하다고 한다. 승객 1명당 개다 수익이 천 원이라고 했을 때 몇 개의 버스 노선이 대상인가? (정답예시:1)

In [1]:
import pandas as pd
from scipy.stats import ttest_ind
from sklearn.cluster import AgglomerativeClustering

In [2]:
df = pd.read_csv("Seoul_Bus_2019.csv")
df.head(2)

,Year_Month,Line_ID,Line_No,Line_Name,Station_ID,Station_Name,H01,H02,H03,H04,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
0,201901,11110364,N13,N13번(송파공영차고지~상계동차고지),110000184,석계역2번출구,0,0,25,33,...,0,0,0,0,0,0,0,0,0,0
1,201901,11110001,100,100번(하계동~용산구청),110000327,한성여객종점,0,0,0,131,...,13,15,21,46,42,66,57,9,6,0


In [3]:
df_info = pd.read_csv("Seoul_Bus_info.csv")
df_info.head(2)

,Bus_no,type
0,N13,심야
1,100,간선


In [4]:
df_join = df.merge(df_info, left_on = "Line_No", right_on= "Bus_no")
df_join.head(2)

,Year_Month,Line_ID,Line_No,Line_Name,Station_ID,Station_Name,H01,H02,H03,H04,...,H17,H18,H19,H20,H21,H22,H23,H24,Bus_no,type
0,201901,11110364,N13,N13번(송파공영차고지~상계동차고지),110000184,석계역2번출구,0,0,25,33,...,0,0,0,0,0,0,0,0,N13,심야
1,201901,11110364,N13,N13번(송파공영차고지~상계동차고지),110000379,석계역1번출구.B,0,0,67,36,...,0,0,0,0,0,0,0,0,N13,심야


In [5]:
df_q1 = df_join.copy()

In [7]:
df_q1["month_total"] = df_q1.loc[:, "H01":"H24"].sum(axis = 1) # 총 승차 숫자 구하기
df_q1.head(2)

,Year_Month,Line_ID,Line_No,Line_Name,Station_ID,Station_Name,H01,H02,H03,H04,...,H18,H19,H20,H21,H22,H23,H24,Bus_no,type,month_total
0,201901,11110364,N13,N13번(송파공영차고지~상계동차고지),110000184,석계역2번출구,0,0,25,33,...,0,0,0,0,0,0,0,N13,심야,58
1,201901,11110364,N13,N13번(송파공영차고지~상계동차고지),110000379,석계역1번출구.B,0,0,67,36,...,0,0,0,0,0,0,0,N13,심야,103


In [10]:
df_q1["type"].unique() # 필터링 하기위해

array(['심야', '간선', '순환', '지선', '마을', '맞춤', '공항', '광역'], dtype=object)

In [12]:
df_q1 = df_q1.loc[df_q1["type"].isin(["지선", "간선"]), ]
df_q1.head(2)

,Year_Month,Line_ID,Line_No,Line_Name,Station_ID,Station_Name,H01,H02,H03,H04,...,H18,H19,H20,H21,H22,H23,H24,Bus_no,type,month_total
3540,201901,11110001,100,100번(하계동~용산구청),110000327,한성여객종점,0,0,0,131,...,46,42,66,57,9,6,0,100,간선,683
3541,201901,11110001,100,100번(하계동~용산구청),110000286,노원평생학습관,0,0,0,0,...,48,21,24,21,25,8,0,100,간선,404


In [13]:
df_q1_agg = df_q1.groupby("Line_No")["month_total"].sum().reset_index()
df_q1_agg.head(2)

,Line_No,month_total
0,100,6597898
1,101,6685417


In [14]:
1000000000 / 1000

1000000.0

In [15]:
df_q1_agg_sub = df_q1_agg.loc[df_q1_agg["month_total"] < 1000000, ]
df_q1_agg_sub.head(2)

,Line_No,month_total
16,1113,770433
17,1114,646939


In [16]:
len(df_q1_agg_sub) # 정답

17

### 2. 간선 버스노선 버스기사들은 간선 버스노선이 지선 버스노선 대비 정류장 개수가 많아 버스 기사 확충 또는 배차 간격 조정을 사측에 요구하고 있다. 
### 경영지원팀은 요구를 수용하기 위해 지선과 간선 노선의 정류장 개수를 파악하고 이 차이가 통계적으로 유의한지 확인해보려 한다. 
### 간선 버스노선의 평균 정류장 개수와 지선 버스노선의 평균 정류장 개수의 평균을 적절한 검정을 통해 그 차이를 비교하고 그 검정통계량의 절대값을 소수점 둘째자리까지 반올림하여 기술하시오. (정답예시: 1.23)

In [28]:
df_q2 = df_join.copy()
df_q2.head(2)

,Year_Month,Line_ID,Line_No,Line_Name,Station_ID,Station_Name,H01,H02,H03,H04,...,H17,H18,H19,H20,H21,H22,H23,H24,Bus_no,type
0,201901,11110364,N13,N13번(송파공영차고지~상계동차고지),110000184,석계역2번출구,0,0,25,33,...,0,0,0,0,0,0,0,0,N13,심야
1,201901,11110364,N13,N13번(송파공영차고지~상계동차고지),110000379,석계역1번출구.B,0,0,67,36,...,0,0,0,0,0,0,0,0,N13,심야


In [29]:
df_q2_gan = df_q2.loc[df_q2["type"] == "간선", ]
df_q2_gee = df_q2.loc[df_q2["type"] == "지선", ]

In [30]:
df_q2_gan.head(2)

,Year_Month,Line_ID,Line_No,Line_Name,Station_ID,Station_Name,H01,H02,H03,H04,...,H17,H18,H19,H20,H21,H22,H23,H24,Bus_no,type
3540,201901,11110001,100,100번(하계동~용산구청),110000327,한성여객종점,0,0,0,131,...,21,46,42,66,57,9,6,0,100,간선
3541,201901,11110001,100,100번(하계동~용산구청),110000286,노원평생학습관,0,0,0,0,...,34,48,21,24,21,25,8,0,100,간선


In [31]:
df_q2_gee.head(2)

,Year_Month,Line_ID,Line_No,Line_Name,Station_ID,Station_Name,H01,H02,H03,H04,...,H17,H18,H19,H20,H21,H22,H23,H24,Bus_no,type
79153,201901,11110077,2230,2230번(면목동~경동시장),990071180,북부운수차고지(가상),0,0,0,0,...,112,119,174,155,83,59,31,0,2230,지선
79154,201901,11110077,2230,2230번(면목동~경동시장),106000089,방약국앞,0,0,0,0,...,111,166,176,174,82,89,87,0,2230,지선


In [32]:
df_q2_gan2 = df_q2_gan[["Line_No", "Station_ID"]].drop_duplicates()
df_q2_gee2 = df_q2_gee[["Line_No", "Station_ID"]].drop_duplicates()

In [36]:
len(df_q2_gan2 )

12594

In [37]:
len(df_q2_gee2)

11808

In [39]:
df_q2_gan2_cnt = df_q2_gan.groupby("Line_No")["Station_ID"].nunique().reset_index() # nunique :number of unique : 고유한 값의 개수
df_q2_gee2_cnt = df_q2_gee.groupby("Line_No")["Station_ID"].nunique().reset_index()
df_q2_gan2_cnt.head()

,Line_No,Station_ID
0,100,121
1,101,83
2,102,69
3,103,71
4,104,67


In [45]:
stat, p = ttest_ind(df_q2_gan2_cnt["Station_ID"],
                    df_q2_gee2_cnt["Station_ID"])
round(abs(stat), 2)

9.5

### 3. 출퇴근 시간 배차간격 조정을 위해 우선적으로 각 정류소별 시간대별 출근 시간 승차 패턴을 파악하고자 한다.
### 지석 버스 노선을 대상으로 각 정류장별 승차인원을 계층적 군집분석을 활용하여 6개의 군집으로 분할하였을 때 출근 시간대에 승차 인원이 가장 많은 정류소 군집의 번호는 몇 번인가? (정답 예시 : 2)
- 출근 시간대는 오전 7시부터 9시까지로 정의
- 군집분석 시 자료는 Min-Max 정규화 실시
- 거리 계산은 유클리디안 거리로하고 유사도는 Ward.D 방법을 사용

In [59]:
df_q3 = df_join.copy()
df_q3.head(2)

,Year_Month,Line_ID,Line_No,Line_Name,Station_ID,Station_Name,H01,H02,H03,H04,...,H17,H18,H19,H20,H21,H22,H23,H24,Bus_no,type
0,201901,11110364,N13,N13번(송파공영차고지~상계동차고지),110000184,석계역2번출구,0,0,25,33,...,0,0,0,0,0,0,0,0,N13,심야
1,201901,11110364,N13,N13번(송파공영차고지~상계동차고지),110000379,석계역1번출구.B,0,0,67,36,...,0,0,0,0,0,0,0,0,N13,심야


In [60]:
df_q3 = df_q3.loc[df_q3["type"] == "지선", ]
df_q3.head(2)

,Year_Month,Line_ID,Line_No,Line_Name,Station_ID,Station_Name,H01,H02,H03,H04,...,H17,H18,H19,H20,H21,H22,H23,H24,Bus_no,type
79153,201901,11110077,2230,2230번(면목동~경동시장),990071180,북부운수차고지(가상),0,0,0,0,...,112,119,174,155,83,59,31,0,2230,지선
79154,201901,11110077,2230,2230번(면목동~경동시장),106000089,방약국앞,0,0,0,0,...,111,166,176,174,82,89,87,0,2230,지선


In [61]:
df_q3 = pd.concat([df_q3.loc[:, ["Year_Month", "Station_ID"]], # 단순화, drop으로 필요없는것만 뺄수도있음
                   df_q3.loc[:, "H01":"H24"]],
                  axis=1)
df_q3.head(2)

,Year_Month,Station_ID,H01,H02,H03,H04,H05,H06,H07,H08,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
79153,201901,990071180,0,0,0,0,29,92,36,41,...,52,108,112,119,174,155,83,59,31,0
79154,201901,106000089,0,0,0,0,27,76,40,49,...,122,149,111,166,176,174,82,89,87,0


In [63]:
df_q3_agg = df_q3.iloc[:, 1:].groupby("Station_ID").sum().reset_index()
df_q3_agg.head(2)

,Station_ID,H01,H02,H03,H04,H05,H06,H07,H08,H09,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
0,100000006,39,0,0,0,161,1339,3901,8980,11317,...,7894,11008,19574,11539,12734,7101,4336,3744,2101,784
1,100000007,84,2,0,0,1102,5502,12087,35430,48382,...,26193,31473,42557,42853,58098,23348,15397,10904,6998,2172


In [64]:
df_q3_melt = df_q3_agg.melt(id_vars = "Station_ID") # wide form => long form으로 정규화 편하게하기위해
df_q3_melt.head(3)

,Station_ID,variable,value
0,100000006,H01,39
1,100000007,H01,84
2,100000008,H01,64


In [65]:
df_q3_melt = df_q3_melt.sort_values(["Station_ID", "variable"], ascending= [False, True])
df_q3_melt.head(3)

,Station_ID,variable,value
5588,999033574,H01,12
11177,999033574,H02,2
16766,999033574,H03,0


In [66]:
def nor_minmax(x):
    result = (x - x.min()) / (x.max() - x.min())
    return result

In [67]:
df_q3_melt["value_nor"] = df_q3_melt.groupby("Station_ID")["value"].transform(nor_minmax) # 정규화한값 넣어주기
df_q3_melt.head()

,Station_ID,variable,value,value_nor
5588,999033574,H01,12,0.444444
11177,999033574,H02,2,0.074074
16766,999033574,H03,0,0.000000
22355,999033574,H04,0,0.000000
27944,999033574,H05,0,0.000000


In [68]:
# clustering 위해서 
df_q3_pivot = pd.pivot(df_q3_melt, index = "Station_ID", columns = "variable", values = "value_nor")
df_q3_pivot.head(2)

variable,H01,H02,H03,H04,H05,H06,H07,H08,H09,H10,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
Station_ID,,,,,,,,,,,,,,,,,,,,,
100000006,0.001992,0.000000,0.0,0.0,0.008225,0.068407,0.199295,0.458772,0.578165,0.504700,...,0.403290,0.562379,1.000000,0.589506,0.650557,0.362777,0.221518,0.191274,0.107336,0.040053
100000007,0.001446,0.000034,0.0,0.0,0.018968,0.094702,0.208045,0.609832,0.832765,0.799769,...,0.450842,0.541723,0.732504,0.737599,1.000000,0.401873,0.265018,0.187683,0.120452,0.037385


In [69]:
model = AgglomerativeClustering(n_clusters=6).fit(df_q3_pivot)

In [75]:
df_q3_pivot["cluster"] = model.labels_
df_q3_pivot.head(2)

variable,Station_ID,H01,H02,H03,H04,H05,H06,H07,H08,H09,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,cluster
0,100000006,0.001992,0.000000,0.0,0.0,0.008225,0.068407,0.199295,0.458772,0.578165,...,0.562379,1.000000,0.589506,0.650557,0.362777,0.221518,0.191274,0.107336,0.040053,0
1,100000007,0.001446,0.000034,0.0,0.0,0.018968,0.094702,0.208045,0.609832,0.832765,...,0.541723,0.732504,0.737599,1.000000,0.401873,0.265018,0.187683,0.120452,0.037385,0


In [76]:
df_q3_pivot = df_q3_pivot.reset_index()
df_q3_pivot.head(2)

variable,index,Station_ID,H01,H02,H03,H04,H05,H06,H07,H08,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,cluster
0,0,100000006,0.001992,0.000000,0.0,0.0,0.008225,0.068407,0.199295,0.458772,...,0.562379,1.000000,0.589506,0.650557,0.362777,0.221518,0.191274,0.107336,0.040053,0
1,1,100000007,0.001446,0.000034,0.0,0.0,0.018968,0.094702,0.208045,0.609832,...,0.541723,0.732504,0.737599,1.000000,0.401873,0.265018,0.187683,0.120452,0.037385,0


In [77]:
df_q3_pivot_melt = df_q3_pivot.melt(id_vars= ["Station_ID", "cluster"])
df_q3_pivot_melt.head(2)

,Station_ID,cluster,variable,value
0,100000006,0,index,0.0
1,100000007,0,index,1.0


In [78]:
df_q3_pivot_melt["rush_hour"] = df_q3_pivot_melt["variable"].isin(["H07", "H08", 'H09']) +0
df_q3_pivot_melt = df_q3_pivot_melt.loc[df_q3_pivot_melt["rush_hour"] == 1, ]
df_q3_pivot_melt = df_q3_pivot_melt.loc[df_q3_pivot_melt["value"] == 1, ]
df_q3_pivot_melt.head()

,Station_ID,cluster,variable,value,rush_hour
40430,107000211,3,H07,1.0,1
40650,108000172,3,H07,1.0,1
40981,110000105,2,H07,1.0,1
41056,110000194,3,H07,1.0,1
42012,114000427,0,H07,1.0,1


In [80]:
len(df_q3_pivot_melt)

2761

In [81]:
len(df_q3_pivot_melt.loc[:, ["Station_ID", "cluster"]].drop_duplicates()) # 중복제거

2756

In [82]:
df_q3_pivot_melt = df_q3_pivot_melt.loc[:, ["Station_ID", "cluster"]]

In [83]:
df_q3_pivot_melt.groupby("cluster")["Station_ID"].nunique().reset_index()

,cluster,Station_ID
0,0,1012
1,1,2
2,2,709
3,3,19
4,4,899
5,5,115


정답 : 0